# Assistente Virtual para Acessibilidade Digital (Protótipo "Conecta Simples")

**Autor:** [Seu Nome Aqui - ou seu Nick da Imersão]
**Projeto para:** Imersão IA Alura + Google Gemini (Maio 2025)

## Descrição do Projeto

Este projeto é um protótipo de um assistente virtual em português, chamado "Conecta Simples", desenvolvido com o objetivo de promover a inclusão digital e facilitar o acesso a tecnologias para pessoas com baixo letramento digital, idosos, ou qualquer indivíduo que enfrente dificuldades em utilizar interfaces digitais complexas.

O assistente utiliza o poder do Google Gemini para compreender linguagem natural simplificada e fornecer respostas e orientações de forma clara, paciente e empática. O foco é ser uma ponte entre o usuário e o mundo digital, tornando tarefas comuns mais acessíveis.

### Funcionalidades Implementadas (Protótipo Colab):

1.  **Comunicação Facilitada:** Permite ao usuário, através de comandos simples, simular o ato de ligar ou enviar mensagens para contatos. O Gemini interpreta a intenção e extrai as informações necessárias.
2.  **Acesso a Serviços Essenciais:** O usuário pode perguntar como realizar serviços comuns (ex: "Como tirar segunda via de conta?"), e o assistente usa o Gemini para fornecer uma explicação passo a passo em linguagem extremamente simples.
3.  **Alerta de Golpes:** O usuário pode colar mensagens suspeitas (SMS, WhatsApp, etc.), e o assistente utiliza o Gemini para analisar a mensagem, informar se parece um golpe, explicar o porquê de forma simples, e dar uma recomendação clara do que fazer.

## Como Executar este Protótipo no Google Colab

1.  **Pré-requisitos:**
    *   Uma conta Google para acessar o Google Colab.
    *   Uma Chave de API do Google Gemini (Google AI Studio).

2.  **Configuração:**
    *   Abra este notebook (`Novo.ipynb`) no Google Colab.
    *   No menu à esquerda do Colab, clique no ícone de uma **chave (🔑) chamado "Secrets"**.
    *   Clique em "+ Adicionar novo secret".
    *   No campo "Nome", digite EXATAMENTE `GOOGLE_API_KEY`.
    *   No campo "Valor", cole a sua Chave de API do Google Gemini.
    *   Marque a caixinha "Acesso ao notebook".

3.  **Execução:**
    *   Execute as células do notebook na ordem, de 1 a 7.
    *   **Célula 1:** Instala a biblioteca do Gemini.
    *   **Célula 2:** Configura a API Key e carrega o modelo Gemini. Verifique se não há erros aqui.
    *   **Célula 3:** Define a função principal de interação com o Gemini.
    *   **Células 4, 5, 6:** Definem as funcionalidades específicas do assistente.
    *   **Célula 7:** Define e (após descomentar a linha `menu_principal()`) executa o menu principal interativo.
    *   Siga as instruções do menu para testar as funcionalidades.

## Tecnologias Utilizadas

*   Python
*   Google Gemini API (através da biblioteca `google-generativeai`)
*   Google Colab (para desenvolvimento e prototipagem)
```    *   **Não se esqueça de substituir `[Seu Nome Aqui - ou seu Nick da Imersão]` pelo seu nome ou identificação.**

In [ ]:
!pip install -q google-generativeai

In [ ]:
# [2] Configura a API Key e inicializa o modelo Gemini

import google.generativeai as genai # Importação recomendada
from google.colab import userdata
import os # Necessário se você quiser continuar setando a variável de ambiente

# Configura a API Key
# A forma como você fez, usando userdata.get() e os.environ, funciona.
# Alternativamente, e de forma mais direta para esta biblioteca, você pode fazer:
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso a partir do Colab Secrets!")
    api_key_configurada = True
except userdata.SecretNotFoundError:
    print("ERRO: Secret 'GOOGLE_API_KEY' não encontrado. Por favor, adicione sua API Key nos Secrets do Colab.")
    api_key_configurada = False
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API Key: {e}")
    api_key_configurada = False

# Inicializa o modelo (APENAS se a API Key foi configurada)
model = None # Inicializa model como None
if api_key_configurada:
    MODEL_ID = 'gemini-1.5-flash-latest'
    try:
        model = genai.GenerativeModel(MODEL_ID)
        print(f"Modelo '{MODEL_ID}' carregado com sucesso.")
    except Exception as e:
        print(f"ERRO ao carregar o modelo '{MODEL_ID}': {e}")
        print("Verifique se o nome do modelo está correto e se sua API Key tem acesso a ele.")
else:
    print("Modelo não será carregado pois a API Key não foi configurada.")

In [ ]:
# [3] Função base para interagir com o modelo Gemini

print("--- Definição da Função de Interação com Gemini ---")

def interagir_com_gemini(prompt_usuario, configuracoes_geracao=None, tentativas=3):
    """
    Envia um prompt para o modelo Gemini carregado e retorna a resposta em texto.

    Args:
        prompt_usuario (str): O texto do prompt a ser enviado ao modelo.
        configuracoes_geracao (genai.types.GenerationConfig, optional): Configurações para a geração de conteúdo.
                                                                     Ex: genai.types.GenerationConfig(temperature=0.7)
                                                                     Se None, usará as configurações padrão.
        tentativas (int): Número de tentativas em caso de erro na API antes de desistir.

    Returns:
        str: A resposta do modelo em formato de texto, ou uma mensagem de erro.
    """
    if model is None: # Verifica se o 'model' foi inicializado na Célula 2
        print("ERRO na função interagir_com_gemini: O modelo Gemini não foi carregado.")
        return "Erro: O modelo Gemini não está disponível. Verifique a Célula 2."

    if not prompt_usuario:
        print("AVISO na função interagir_com_gemini: Prompt do usuário está vazio.")
        return "Aviso: Nenhum prompt fornecido."

    print(f"INFO: Enviando prompt ao Gemini: \"{prompt_usuario[:100]}...\"") # Mostra os primeiros 100 caracteres do prompt

    for tentativa in range(tentativas):
        try:
            if configuracoes_geracao:
                response = model.generate_content(
                    prompt_usuario,
                    generation_config=configuracoes_geracao
                )
            else:
                response = model.generate_content(prompt_usuario)

            # Verifica se há candidatos e se o primeiro candidato tem conteúdo
            if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
                resposta_texto = "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))
                print(f"INFO: Resposta recebida do Gemini na tentativa {tentativa + 1}.")
                return resposta_texto
            else:
                # Caso a resposta não tenha o formato esperado (ex: bloqueio de segurança sem detalhes claros via 'text')
                print(f"AVISO: Resposta do Gemini na tentativa {tentativa + 1} não continha texto utilizável ou foi bloqueada.")
                # Você pode querer inspecionar 'response.prompt_feedback' para bloqueios
                if response.prompt_feedback:
                    print(f"  Feedback do prompt: {response.prompt_feedback}")
                if not response.candidates:
                     print("  Nenhum candidato na resposta.")
                # Se for a última tentativa, retorna uma mensagem mais genérica de falha
                if tentativa == tentativas - 1:
                    return "Erro: O modelo Gemini não retornou um conteúdo utilizável após várias tentativas."
                # Continua para a próxima tentativa

        except Exception as e:
            print(f"ERRO na função interagir_com_gemini (tentativa {tentativa + 1}/{tentativas}): {e}")
            if tentativa == tentativas - 1: # Se for a última tentativa
                return f"Erro: Falha ao gerar conteúdo após {tentativas} tentativas. Último erro: {e}"
            # Espera um pouco antes de tentar novamente (opcional, mas bom para erros transientes)
            # import time
            # time.sleep(1)

    return "Erro: Falha ao obter resposta do modelo após todas as tentativas." # Caso o loop termine sem sucesso


# Teste rápido opcional da função (descomente para testar após executar a célula)
# print("\n--- Teste da Função de Interação ---")
# if model: # Verifica se o modelo foi carregado
#     prompt_teste = "Olá, Gemini! Qual é a capital da França? Responda de forma concisa."
#     resposta_teste = interagir_com_gemini(prompt_teste)
#     print(f"\nPrompt de Teste: {prompt_teste}")
#     print(f"Resposta do Teste: {resposta_teste}")
#
#     # Teste com configuração de geração (ex: temperatura)
#     # config_baixa_temp = genai.types.GenerationConfig(temperature=0.1)
#     # resposta_teste_baixa_temp = interagir_com_gemini("Conte uma piada muito curta.", configuracoes_geracao=config_baixa_temp)
#     # print(f"\nResposta do Teste (baixa temperatura): {resposta_teste_baixa_temp}")
# else:
#     print("AVISO: Teste da função de interação não pode ser executado pois o modelo não foi carregado.")

print("--- Fim da Definição da Função de Interação ---")

In [ ]:
# [4] Funcionalidade: Comunicação Facilitada (Simulação)

print("--- Definição da Funcionalidade: Comunicação Facilitada ---")

def processar_comando_comunicacao():
    """
    Processa comandos de comunicação do usuário (ligar, enviar mensagem)
    usando o Gemini para interpretar a intenção e extrair informações.
    Simula a execução da ação.
    """
    # Verifica se a API Key e o modelo estão prontos (definidos nas Células anteriores)
    if not api_key_configurada: # 'api_key_configurada' deve ter sido definida na Célula 2
        print("Assistente: Desculpe, não posso processar comandos de comunicação. A API Key não está configurada.")
        return
    if model is None: # 'model' deve ter sido inicializado na Célula 2
        print("Assistente: Desculpe, não posso processar comandos de comunicação. O modelo de IA não está pronto.")
        return

    print("\n--- Testando Funcionalidade: Comunicação Facilitada ---")
    comando_usuario = input("Assistente: Olá! Sou seu ajudante digital. Você quer ligar para alguém ou mandar uma mensagem?\n"
                            "            Por exemplo, diga: 'Ligar para Ana' ou 'Mandar mensagem para José: estou a caminho'.\n"
                            "Usuário: ")

    if not comando_usuario:
        print("Assistente: Você não disse nada. Se precisar de ajuda, é só chamar!")
        return

    # Prompt detalhado para o Gemini
    prompt_gemini = f"""
    Você é um assistente virtual extremamente paciente, amigável e que usa linguagem muito simples.
    Seu objetivo é ajudar pessoas com pouca familiaridade com tecnologia a realizar tarefas de comunicação.

    Analise o comando do usuário abaixo. Sua tarefa é identificar:
    1. A AÇÃO: Deve ser "Ligar" ou "Enviar Mensagem".
    2. O CONTATO: O nome da pessoa para quem a ação se destina.
    3. A MENSAGEM: O conteúdo da mensagem, APENAS se a ação for "Enviar Mensagem".

    Formato da sua resposta:
    Se todas as informações estiverem claras, responda estritamente no seguinte formato:
    AÇÃO: [Ligar ou Enviar Mensagem]
    CONTATO: [Nome do Contato]
    MENSAGEM: [Texto da Mensagem, APENAS se a ação for Enviar Mensagem, caso contrário deixe esta linha em branco ou omita]

    Se alguma informação estiver faltando ou não estiver clara (por exemplo, se o usuário disser apenas "Ligar" sem dizer para quem,
    ou "Mandar mensagem" sem o destinatário ou o conteúdo), faça UMA pergunta MUITO SIMPLES e DIRETA para obter a informação que falta.
    Não tente adivinhar. Não prossiga com a ação incompleta.

    Exemplos de perguntas de esclarecimento:
    - "Para quem você gostaria de ligar?"
    - "Para quem é a mensagem?"
    - "O que você gostaria de dizer na mensagem para [Nome do Contato]?"

    Comando do usuário: "{comando_usuario}"

    Sua análise:
    """

    print("Assistente: Entendido. Vou analisar seu pedido...")
    # Chama a função da Célula 3 para interagir com o Gemini
    # Podemos ajustar a temperatura para respostas mais diretas e menos criativas para esta tarefa
    config_comunicacao = genai.types.GenerationConfig(temperature=0.2)
    resposta_gemini = interagir_com_gemini(prompt_gemini, configuracoes_geracao=config_comunicacao)

    print(f"\nAssistente (Análise do Gemini para o desenvolvedor):\n{resposta_gemini}\n") # Mostra a resposta crua do Gemini

    # Simulação da ação baseada na resposta do Gemini
    # Esta parte faz um parsing simples da resposta. Em um app real, seria mais robusto.
    if "ERRO:" in resposta_gemini or "Erro:" in resposta_gemini : # Se a função interagir_com_gemini retornou erro
        print(f"Assistente: Desculpe, tive um problema ao tentar entender seu pedido. Detalhe: {resposta_gemini}")
    elif "?" in resposta_gemini: # Se o Gemini fez uma pergunta de esclarecimento
        print(f"Assistente: {resposta_gemini}") # A resposta do Gemini já é a pergunta para o usuário
    elif "AÇÃO:" in resposta_gemini:
        acao = ""
        contato = ""
        mensagem = ""

        linhas_resposta = resposta_gemini.strip().split('\n')
        for linha in linhas_resposta:
            if linha.startswith("AÇÃO:"):
                acao = linha.replace("AÇÃO:", "").strip()
            elif linha.startswith("CONTATO:"):
                contato = linha.replace("CONTATO:", "").strip()
            elif linha.startswith("MENSAGEM:"):
                mensagem = linha.replace("MENSAGEM:", "").strip()

        if acao == "Ligar" and contato:
            print(f"Assistente (Simulação): Entendido! Agora eu iria iniciar uma ligação para {contato}.")
            print(f"                  (Imagine o celular discando para {contato}...)")
        elif acao == "Enviar Mensagem" and contato:
            if mensagem:
                print(f"Assistente (Simulação): Entendido! Agora eu iria enviar a mensagem '{mensagem}' para {contato}.")
                print(f"                  (Imagine a mensagem sendo enviada para {contato}...)")
            else: # Gemini identificou "Enviar Mensagem" mas não extraiu a mensagem (deveria ter perguntado)
                print(f"Assistente: Eu entendi que você quer enviar uma mensagem para {contato}, mas não ficou claro o que você quer dizer. Pode me dizer a mensagem?")
        elif acao and not contato: # Gemini identificou ação mas não contato (deveria ter perguntado)
             print(f"Assistente: Entendi que você quer '{acao}', mas não sei para quem. Pode me dizer o nome da pessoa?")
        else:
            print("Assistente: Entendi algumas coisas, mas não ficou claro para mim o que fazer. Poderia tentar de novo de uma forma diferente?")
            print(f"             (O que eu entendi: {resposta_gemini})") # Mostra o que entendeu se não conseguiu parsear
    else:
        print(f"Assistente: Hum, não consegui entender seu pedido da forma que esperava. Pode tentar de novo? (Resposta que obtive: {resposta_gemini})")

# --- Como Testar esta Funcionalidade ---
# 1. Certifique-se de que a Célula 2 (Configuração da API e Modelo) e a Célula 3 (interagir_com_gemini) foram executadas sem erros.
# 2. Descomente a linha abaixo e execute ESTA CÉLULA (Célula 4).
# 3. Quando solicitado, digite um comando como "Ligar para mamãe" ou "Mandar zap para João dizendo que estou chegando".

# print("\nPara testar, descomente a linha abaixo e execute a célula:")
# processar_comando_comunicacao()

print("--- Fim da Definição da Funcionalidade: Comunicação Facilitada ---")

In [ ]:
# [5] Funcionalidade: Acesso a Serviços Essenciais (Busca e Simplificação de Informação)

print("--- Definição da Funcionalidade: Acesso a Serviços Essenciais ---")

def buscar_e_simplificar_informacao_servico(): # Adicione o '_servico' aqui
    """
    Permite ao usuário perguntar sobre como acessar serviços ou obter informações.
    Usa o Gemini para buscar (simuladamente) e simplificar a informação em linguagem acessível.
    """
    # Verifica se a API Key e o modelo estão prontos
    if not api_key_configurada:
        print("Assistente: Desculpe, não posso te ajudar com isso agora. A API Key não está configurada.")
        return
    if model is None:
        print("Assistente: Desculpe, não posso te ajudar com isso agora. O modelo de IA não está pronto.")
        return

    print("\n--- Testando Funcionalidade: Acesso a Serviços Essenciais ---")
    pergunta_usuario = input("Assistente: Olá! Posso te ajudar a entender como fazer alguma coisa ou onde encontrar um serviço.\n"
                             "            Por exemplo, pergunte: 'Como faço para tirar a segunda via da minha conta de luz?'\n"
                             "            ou 'Onde posso me vacinar contra a gripe?'\n"
                             "Usuário: ")

    if not pergunta_usuario:
        print("Assistente: Você não perguntou nada. Se tiver alguma dúvida, é só chamar!")
        return

    # Prompt detalhado para o Gemini
    prompt_gemini = f"""
    Você é um assistente virtual muito prestativo e didático. Sua especialidade é explicar coisas
    que parecem complicadas de um jeito MUITO FÁCIL de entender, como se estivesse conversando
    com uma pessoa idosa ou alguém que tem muita dificuldade com tecnologia e leitura.

    O usuário tem a seguinte dúvida ou pergunta: "{pergunta_usuario}"

    Sua tarefa é:
    1. Entender a pergunta do usuário.
    2. Se for uma pergunta sobre como fazer algo (um procedimento, um serviço), explique em PASSOS CURTOS e CLAROS. Use.

    O usuário tem a seguinte dúvida sobre um serviço:
    "{pergunta_usuario}"

    Sua tarefa é:
    1. Entender a necessidade do usuário.
    2. Fornecer uma explicação passo a passo, como se estivesse conversando com uma criança ou um amigo que não entende nada de burocracia.
    3. Usar frases MUITO CURTAS e palavras MUITO SIMPLES do dia a dia.
    4. EVITAR AO MÁXIMO qualquer jargão, sigla ou termo técnico. Se for impossível evitar, explique o termo de forma muito básica IMEDIATAMENTE.
    5. Dividir a explicação em PASSOS CLAROS (Ex: Passo 1:, Passo 2:). Se for algo muito simples, pode ser um parágrafo curto.
    6. Ser objetivo e focar apenas no essencial para que o usuário consiga realizar a tarefa.
    7. Se a pergunta for muito vaga ou você não tiver certeza do que o usuário precisa, faça UMA pergunta simples para esclarecer.
       Ex: "Para te ajudar melhor, você poderia me dizer de qual cidade você está falando para a conta de água?" (se relevante e não fornecido).
    8. Não invente informações. Se você não souber responder com segurança, diga de forma simples que não encontrou essa informação.
       Ex: "Hum, sobre isso eu não encontrei uma informação clara para te passar agora."

    Formato da sua resposta:
    Comece com uma frase curta e amigável, e depois siga com os passos ou a explicação.

    Explicação Simplificada:
    """

    print("Assistente: Entendido! Vou procurar a informação e te explicar de um jeito fácil...")
    # Chama a função da Célula 3 para interagir com o Gemini
    # Temperatura um pouco mais alta pode permitir explicações mais naturais, mas ainda focadas.
    config_servicos = genai.types.GenerationConfig(temperature=0.5)
    resposta_gemini = interagir_com_gemini(prompt_gemini, configuracoes_geracao=config_servicos)

    print(f"\nAssistente (Explicação do Gemini):\n{resposta_gemini}\n")

    if "ERRO:" in resposta_gemini or "Erro:" in resposta_gemini:
        print(f"Assistente: Desculpe, tive um problema ao tentar buscar essa informação. Detalhe: {resposta_gemini}")
    # Não há muito parsing a ser feito aqui, pois a resposta do Gemini já deve ser a explicação formatada.

# --- Como Testar esta Funcionalidade ---
# 1. Certifique-se de que a Célula 2, 3 e 4 foram executadas.
# 2. Descomente a linha abaixo e execute ESTA CÉLULA (Célula 5).
# 3. Quando solicitado, faça uma pergunta como:
#    - "Como renovar minha carteira de identidade?"
#    - "O que preciso para tirar o CPF?"
#    - "Como agendar uma consulta no posto de saúde?"
#    - "Como bloquear meu celular roubado?"

# print("\nPara testar, descomente a linha abaixo e execute a célula:")
# buscar_e_simplificar_informacao_servico()

print("--- Fim da Definição da Funcionalidade: Acesso a Serviços Essenciais ---")

In [ ]:
# [6] Funcionalidade: Navegação Assistida e Segura (Alerta de Golpes)

print("--- Definição da Funcionalidade: Alerta de Golpes ---")

def analisar_mensagem_suspeita():
    """
    Permite ao usuário colar uma mensagem suspeita e usa o Gemini para
    analisá-la, alertar sobre possíveis golpes e dar recomendações simples.
    """
    # Verifica se a API Key e o modelo estão prontos
    if not api_key_configurada:
        print("Assistente: Desculpe, não posso analisar mensagens agora. A API Key não está configurada.")
        return
    if model is None:
        print("Assistente: Desculpe, não posso analisar mensagens agora. O modelo de IA não está pronto.")
        return

    print("\n--- Testando Funcionalidade: Alerta de Golpes ---")
    mensagem_usuario = input("Assistente: Se você recebeu uma mensagem que parece estranha ou um golpe (pelo celular ou computador),\n"
                             "            copie a mensagem inteira e cole aqui para eu dar uma olhada e te ajudar.\n"
                             "Usuário (cole a mensagem aqui): ")

    if not mensagem_usuario:
        print("Assistente: Você não colou nenhuma mensagem. Se precisar de ajuda com alguma mensagem, é só me mostrar!")
        return

    # Prompt detalhado para o Gemini
    prompt_gemini = f"""
    Você é um especialista em segurança digital que fala de forma EXTREMAMENTE SIMPLES, como se estivesse alertando um
    amigo querido ou um familiar idoso sobre perigos na internet. Seja muito direto e use palavras fáceis.

    O usuário recebeu a seguinte mensagem e está desconfiado:
    ---
    {mensagem_usuario}
    ---

    Sua tarefa é analisar esta mensagem e responder ao usuário da seguinte forma:

    1.  **PARECE GOLPE?**: Responda com "SIM, PARECE MUITO UM GOLPE!" ou "NÃO, NÃO PARECE SER UM GOLPE." ou "HUM, É UM POUCO SUSPEITO, FIQUE ATENTO.". Seja bem direto aqui.
    2.  **POR QUÊ?**: Explique em POUCAS PALAVRAS e de forma MUITO SIMPLES o motivo da sua conclusão. Use exemplos fáceis.
        (Exemplos de motivos: "Pede dinheiro muito rápido.", "Tem um link esquisito que não é do banco.", "Promete uma coisa boa demais para ser verdade.", "Tem muitos erros de português.", "Pede sua senha ou código pessoal.")
    3.  **O QUE FAZER?**: Dê uma recomendação MUITO CLARA e SIMPLES sobre o que o usuário deve fazer.
        (Exemplos de recomendações: "NÃO CLIQUE nesse link de jeito nenhum.", "APAGUE essa mensagem agora mesmo.", "NÃO responda e bloqueie o número.", "Se ficou na dúvida, ligue para o seu banco pelo número que está no seu cartão, não por este aqui.")

    Formato da sua resposta (siga este formato estritamente):
    PARECE GOLPE?: [Sua conclusão direta aqui]
    POR QUÊ?: [Sua explicação simples aqui]
    O QUE FAZER?: [Sua recomendação clara aqui]

    Lembre-se: Linguagem super simples, direta e protetora.
    """

    print("Assistente: Entendido. Vou analisar essa mensagem com cuidado...")
    # Para esta tarefa, queremos uma resposta bem direta e factual.
    config_alerta = genai.types.GenerationConfig(temperature=0.1, top_p=0.8)
    resposta_gemini = interagir_com_gemini(prompt_gemini, configuracoes_geracao=config_alerta)

    print(f"\nAssistente (Análise do Gemini para o desenvolvedor):\n{resposta_gemini}\n") # Mostra a resposta crua

    # Extrai e apresenta a resposta de forma mais amigável
    if "ERRO:" in resposta_gemini or "Erro:" in resposta_gemini:
        print(f"Assistente: Desculpe, tive um problema ao tentar analisar essa mensagem. Detalhe: {resposta_gemini}")
    elif "PARECE GOLPE?:" in resposta_gemini:
        parecer = ""
        porque = ""
        oque_fazer = ""

        linhas_resposta = resposta_gemini.strip().split('\n')
        for linha in linhas_resposta:
            if linha.startswith("PARECE GOLPE?:"):
                parecer = linha.replace("PARECE GOLPE?:", "").strip()
            elif linha.startswith("POR QUÊ?:"):
                porque = linha.replace("POR QUÊ?:", "").strip()
            elif linha.startswith("O QUE FAZER?:"):
                oque_fazer = linha.replace("O QUE FAZER?:", "").strip()

        print("Assistente (Minha Análise):")
        if parecer:
            print(f"  🚨 {parecer}")
        if porque:
            print(f"  🤔 Motivo: {porque}")
        if oque_fazer:
            print(f"  👉 Recomendação: {oque_fazer}")

        if not parecer and not porque and not oque_fazer: # Se o parsing falhou
             print(f"Assistente: Analisei a mensagem, mas não consegui formatar a resposta como esperado. Veja a análise completa:\n{resposta_gemini}")
    else:
        print(f"Assistente: Hum, analisei a mensagem, mas o resultado não veio no formato que eu esperava. Veja o que recebi:\n{resposta_gemini}")


# --- Como Testar esta Funcionalidade ---
# 1. Certifique-se de que as Células 2, 3, 4 e 5 foram executadas.
# 2. Descomente a linha abaixo e execute ESTA CÉLULA (Célula 6).
# 3. Quando solicitado, cole mensagens suspeitas. Você pode procurar exemplos de mensagens de golpe na internet para testar.
#    Exemplos de frases/mensagens para testar:
#    - "Parabéns! Você ganhou um prêmio de R$5000! Clique aqui para resgatar: bit.ly/premiofake"
#    - "Seu CPF será cancelado! Regularize sua situação URGENTE acessando br-gov-atualiza.com"
#    - "Olá mãe, meu celular quebrou, estou usando este número provisório. Preciso pagar uma conta urgente, pode me fazer um PIX de R$350?"
#    - "Banco XYZ: Detectamos uma compra suspeita de R$1250 no seu cartão final 4004. Se não reconhece, ligue URGENTE para 0800-XXXX-YYYY." (onde o 0800 é falso)
#    - "Sua fatura da NETFLIX venceu, pague agora para não perder o acesso: meupagamentoflix.net"

# print("\nPara testar, descomente a linha abaixo e execute a célula:")
# analisar_mensagem_suspeita()

print("--- Fim da Definição da Funcionalidade: Alerta de Golpes ---")

In [ ]:
print("--- Definição do Menu Principal ---")

def menu_principal():
    """
    Apresenta um menu para o usuário escolher qual funcionalidade do assistente virtual testar.
    """
    # Verifica se a API Key e o modelo estão prontos (essencial para qualquer funcionalidade)
    if not api_key_configurada: # 'api_key_configurada' da Célula 2
        print("Assistente: ERRO CRÍTICO - Não posso iniciar. A API Key do Google Gemini não está configurada.")
        print("            Por favor, verifique a Célula 2 e execute-a novamente após configurar seus Secrets no Colab.")
        return
    if model is None: # 'model' da Célula 2
        print("Assistente: ERRO CRÍTICO - Não posso iniciar. O modelo de IA não foi carregado corretamente.")
        print("            Por favor, verifique os logs da Célula 2 para entender o motivo.")
        return

    print("\n===================================================================")
    print("  Bem-vindo ao Protótipo do Assistente Virtual para Acessibilidade!")
    print("===================================================================")
    print("  Este assistente foi desenhado para ser simples e útil.")

    while True:
        print("\n--- MENU PRINCIPAL ---")
        print("Qual ajuda você gostaria de testar hoje?")
        print("  1: 🗣️  Comunicação Facilitada (Simular Ligar ou Enviar Mensagem)")
        print("  2: ℹ️  Acesso a Serviços (Entender como fazer algo)")
        print("  3: 🛡️  Alerta de Golpes (Analisar mensagem suspeita)")
        print("  --------------------")
        print("  4: 🚪 Sair do Assistente")

        escolha = input("Digite o número da sua escolha (1, 2, 3 ou 4): ")

        if escolha == '1':
            print("\n--- Iniciando: Comunicação Facilitada ---")
            # Chama a função da Célula 4
            try:
                processar_comando_comunicacao()
            except NameError:
                print("ERRO: A função 'processar_comando_comunicacao' não foi definida. Execute a Célula 4.")
            except Exception as e:
                print(f"Ocorreu um erro inesperado ao executar a comunicação facilitada: {e}")
        elif escolha == '2':
            print("\n--- Iniciando: Acesso a Serviços ---")
            # Chama a função da Célula 5
            try:
                buscar_e_simplificar_informacao_servico() # Certifique-se que o nome da função está correto
            except NameError:
                print("ERRO: A função 'buscar_e_simplificar_informacao_servico' não foi definida. Execute a Célula 5.")
            except Exception as e:
                print(f"Ocorreu um erro inesperado ao executar o acesso a serviços: {e}")
        elif escolha == '3':
            print("\n--- Iniciando: Alerta de Golpes ---")
            # Chama a função da Célula 6
            try:
                analisar_mensagem_suspeita()
            except NameError:
                print("ERRO: A função 'analisar_mensagem_suspeita' não foi definida. Execute a Célula 6.")
            except Exception as e:
                print(f"Ocorreu um erro inesperado ao executar o alerta de golpes: {e}")
        elif escolha == '4':
            print("\nAssistente: Entendido! Espero ter ajudado. Até a próxima!")
            print("===================================================================")
            break  # Sai do loop while, terminando o menu
        else:
            print("\nAssistente: 🤔 Hum, não entendi essa opção. Por favor, digite apenas o número 1, 2, 3 ou 4.")

        print("\n-------------------------------------------------------------------")
        input("Pressione Enter para voltar ao Menu Principal...") # Pausa para o usuário ler a saída

# --- Como Usar o Menu Principal ---
# 1. Certifique-se de que TODAS as células anteriores (1 a 6) foram executadas
#    na ordem correta e sem erros críticos (especialmente Célula 2 e 3).
# 2. Descomente a linha abaixo e execute ESTA CÉLULA (Célula 7).
# 3. O menu será exibido e você poderá interagir com as funcionalidades.

# print("\nPara iniciar o Assistente Virtual, descomente a linha abaixo e execute a célula:")
# menu_principal()

print("--- Fim da Definição do Menu Principal ---")# [7] Menu Principal para Testes e Demonstração

print("--- Definição do Menu Principal ---")

def menu_principal():
    """
    Apresenta um menu para o usuário escolher qual funcionalidade do assistente virtual testar.
    """
    # Verifica se a API Key e o modelo estão prontos (essencial para qualquer funcionalidade)
    if not api_key_configurada: # 'api_key_configurada' da Célula 2
        print("Assistente: ERRO CRÍTICO - Não posso iniciar. A API Key do Google Gemini não está configurada.")
        print("            Por favor, verifique a Célula 2 e execute-a novamente após configurar seus Secrets no Colab.")
        return
    if model is None: # 'model' da Célula 2
        print("Assistente: ERRO CRÍTICO - Não posso iniciar. O modelo de IA não foi carregado corretamente.")
        print("            Por favor, verifique os logs da Célula 2 para entender o motivo.")
        return

    print("\n===================================================================")
    print("  Bem-vindo ao Protótipo do Assistente Virtual para Acessibilidade!")
    print("===================================================================")
    print("  Este assistente foi desenhado para ser simples e útil.")

    while True:
        print("\n--- MENU PRINCIPAL ---")
        print("Qual ajuda você gostaria de testar hoje?")
        print("  1: 🗣️  Comunicação Facilitada (Simular Ligar ou Enviar Mensagem)")
        print("  2: ℹ️  Acesso a Serviços (Entender como fazer algo)")
        print("  3: 🛡️  Alerta de Golpes (Analisar mensagem suspeita)")
        print("  --------------------")
        print("  4: 🚪 Sair do Assistente")

        escolha = input("Digite o número da sua escolha (1, 2, 3 ou 4): ")

        if escolha == '1':
            print("\n--- Iniciando: Comunicação Facilitada ---")
            # Chama a função da Célula 4
            try:
                processar_comando_comunicacao()
            except NameError:
                print("ERRO: A função 'processar_comando_comunicacao' não foi definida. Execute a Célula 4.")
            except Exception as e:
                print(f"Ocorreu um erro inesperado ao executar a comunicação facilitada: {e}")
        elif escolha == '2':
            print("\n--- Iniciando: Acesso a Serviços ---")
            # Chama a função da Célula 5
            try:
                buscar_e_simplificar_informacao_servico() # Certifique-se que o nome da função está correto
            except NameError:
                print("ERRO: A função 'buscar_e_simplificar_informacao_servico' não foi definida. Execute a Célula 5.")
            except Exception as e:
                print(f"Ocorreu um erro inesperado ao executar o acesso a serviços: {e}")
        elif escolha == '3':
            print("\n--- Iniciando: Alerta de Golpes ---")
            # Chama a função da Célula 6
            try:
                analisar_mensagem_suspeita()
            except NameError:
                print("ERRO: A função 'analisar_mensagem_suspeita' não foi definida. Execute a Célula 6.")
            except Exception as e:
                print(f"Ocorreu um erro inesperado ao executar o alerta de golpes: {e}")
        elif escolha == '4':
            print("\nAssistente: Entendido! Espero ter ajudado. Até a próxima!")
            print("===================================================================")
            break  # Sai do loop while, terminando o menu
        else:
            print("\nAssistente: 🤔 Hum, não entendi essa opção. Por favor, digite apenas o número 1, 2, 3 ou 4.")

        print("\n-------------------------------------------------------------------")
        input("Pressione Enter para voltar ao Menu Principal...") # Pausa para o usuário ler a saída

# --- Como Usar o Menu Principal ---
# 1. Certifique-se de que TODAS as células anteriores (1 a 6) foram executadas
#    na ordem correta e sem erros críticos (especialmente Célula 2 e 3).
# 2. Descomente a linha abaixo e execute ESTA CÉLULA (Célula 7).
# 3. O menu será exibido e você poderá interagir com as funcionalidades.

# print("\nPara iniciar o Assistente Virtual, descomente a linha abaixo e execute a célula:")
menu_principal()

print("--- Fim da Definição do Menu Principal ---")